<a href="https://colab.research.google.com/github/rynwhd/Language-Models/blob/main/Article_Spinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
import textwrap #doesn't let the paragraph go out of screen 
nltk.download('punkt')
from nltk import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv


--2023-02-10 01:38:33--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  5.07MB/s    in 1.0s    

2023-02-10 01:38:35 (5.07 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [ ]:
df=pd.read_csv('bbc_text_cls.csv')

In [ ]:
df2=pd.read_csv('bbc_text_cls.csv')

In [ ]:
df2=df2[df2['labels']=='business']

In [ ]:
df2['text']=df2['text'].apply(lambda x:x.lower())
df2['text']=df2['text'].apply(lambda x:x.replace("'s",'s'))

In [ ]:
df=df[df['labels']=='business']

In [ ]:
# def my_func(t):
#   ans=''
#   for i in range(len(t)-1):
#     if t[i]+t[i+1]=='\n\n':
#       ans+=' '
#       i+=2
#     else:
#       ans+=t[i].lower()
#   return ans

In [ ]:
df['text']=df['text'].apply(lambda x:x.replace('\n\n',' ').lower())


In [ ]:
df['text']=df['text'].apply(lambda x:x.replace("'s",'s'))

In [ ]:
initial={}
second_order={}

In [ ]:
def add2dic(d,k,v):
  if k not in d:
    d[k]={}
  if v not in d[k].keys():
    d[k][v]=1
  else:
    d[k][v]+=1

In [ ]:
for line in df['text']:
  tokens=word_tokenize(line)
  for i in range(len(tokens)-1):
    t=tokens[i]
    if i==0:
      initial[t]=initial.get(t,0.)+1
    if i==len(tokens)-2:
      w2='END'
    elif i<len(tokens)-2:
      w2=tokens[i+2]
    w1=tokens[i+1]  
    add2dic(second_order,(t,w2),w1)

In [ ]:
for k,v in second_order.items():
  len=sum(v.values())
  for i,j in v.items():
    v[i]=j/len

In [ ]:
import string 

In [ ]:
def sample_word(d):
  p0=np.random.random()
  cumulative=0
  for t,v in d.items():
    cumulative+=v
    if p0<=cumulative:
      return t
  print(d)
  print(p0)
  assert(False) #loop should never get here 

In [ ]:
# def spin_line(line):
#   tokens=word_tokenize(line)
#   T=length(tokens)
#   output=[tokens[0]]
#   rayan=0
#   for i in range(1,T-2):
#     w0=tokens[i]
#     w2=tokens[i+2]
#     key=(w0,w2)
#     if key in second_order:
#       p_dist=second_order[key]
#       if length(p_dist)>1 and np.random.random()<0.3: #controls how often the word is replaced
#         w1=sample_word(p_dist)
#         output.append(tokens[i+1])
#         output.append('<'+w1+'>')
#         i+=1
#   output.append(tokens[-1])
#   for i in output:
#     print(i,end='')
#   # return detokenizer.detokenize(output)


In [ ]:
def spin_line(line):
  tokens=word_tokenize(line)
  i=0
  output=[tokens[0]]
  while i<(length(tokens)-2):
    t_0=tokens[i]
    t_1=tokens[i+1]
    t_2=tokens[i+2]
    key=(t_0,t_2)
    p_dist=second_order[key]
    if length(p_dist)>1:
      middle=sample_word(p_dist)
      output.append(middle)
      # output.append('<'+middle+'>')
      output.append(t_2)
      i+=2
    else:
      output.append(t_1)
      i+=1
  if i==length(tokens)-2:
    output.append(tokens[-1])
  return detokenizer.detokenize(output)


In [ ]:
def spin_text(doc):
  lines=doc.split('\n')
  output=[]
  for line in lines:
    if line:
      new_line=spin_line(line)
    else:
      new_line=line 
    output.append(line)
  return '\n'.join(output)

In [ ]:
np.random.seed(1234)

In [ ]:
test_doc=df2['text'].iloc[0]
print(test_doc.split('\n')[6])

time warner said on friday that it now owns 8% of search-engine google. but its own internet business, aol, had has mixed fortunes. it lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. however, the company said aols underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aols existing customers for high-speed broadband. timewarner also has to restate 2000 and 2003 results following a probe by the us securities exchange commission (sec), which is close to concluding.


In [ ]:
def length(d):
  c=0
  for i in d:
    c+=1
  return c


In [ ]:
new_doc=spin_text(test_doc.split('\n')[6])
new_doc

'time warner said on friday that it now owns 8% of search-engine google. but its own internet business, aol, had has mixed fortunes. it lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. however, the company said aols underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aols existing customers for high-speed broadband. timewarner also has to restate 2000 and 2003 results following a probe by the us securities exchange commission (sec), which is close to concluding.'

In [ ]:
print(textwrap.fill(new_doc,replace_whitespace=False,fix_sentence_endings=True))

time warner said on friday that it now owns 8% of search-engine
google.  but its own internet business, aol, had has mixed fortunes.
it lost 464,000 subscribers in the fourth quarter profits were lower
than in the preceding three quarters.  however, the company said aols
underlying profit before exceptional items rose 8% on the back of
stronger internet advertising revenues.  it hopes to increase
subscribers by offering the online service free to timewarner internet
customers and will try to sign up aols existing customers for high-
speed broadband.  timewarner also has to restate 2000 and 2003 results
following a probe by the us securities exchange commission (sec),
which is close to concluding.


'british hurdler sarah claxton is confident she can win her first major medal at next months european indoor championships in madrid.'

In [ ]:
# text="i am using this string to make a new sentence. can you do this?"
# sent=[]
# t=word_tokenize(text)
# T=0
# for i in t:
#   T+=1
# print(T)
# # T=len(t)
# for j in range(T-2):
#   if j==T-2:
#     w2='END'
#   else: 
#     w2=t[j+2]
#   w0=t[j]
#   if (w0,w2) in second_order:
#     max=0
#     for i,v in second_order[(w0,w2)].items():
#       if v>max:
#         max=v
#     for i,v in second_order.items():
#       if v==max:
#         max=i
#     w1=max
#   else:
#     w1=t[j+1]
#     sent.append(w0)
#     sent.append(w1)
#     sent.append(w2)
#   if w2 in string.punctuation:
#     j+=2
# print(sent)

16
['i', 'am', 'using', 'using', 'this', 'string', 'string', 'to', 'make', 'make', 'a', 'new', 'sentence', '.', 'can', 'can', 'you', 'do', 'you', 'do', 'this', 'do', 'this', '?']
